## Dataset of Movies
- Scrape link : https://www.themoviedb.org/movie
- get  title, release date, cost, url, genres(action/horror), runtime

## Page load with lib
- get the page with `request`
- Use `Bs4` for webpage Process
- Collect the info
- Create the func
- Create `CSV` with that info

libs

In [ ]:
import requests
from bs4 import BeautifulSoup
import pprint
import pandas as pd

## scrape a single movie
- release date, genres, runtime
- movie link: https://www.themoviedb.org/movie/460465

In [ ]:
url = 'https://www.themoviedb.org/movie/460465'
resp = requests.get(url)
html_doc = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
release_date_tag = html_doc.select_one('.release')
release_date = release_date_tag.text.strip().split(' ')[0]
release_date

'04/23/2021'

In [ ]:
genres_tag = html_doc.select_one('.genres')
genres = genres_tag.text.strip().replace("\xa0", "") 
genres

'Action,Fantasy,Adventure,Science Fiction'

In [ ]:
runtime_tag = html_doc.select_one('.runtime')
runtime = runtime_tag.text.strip()
runtime

'1h 50m'

In [ ]:
budget_tag = html_doc.find("section",{'class': 'facts left_column'}).find_all('p')
budget = budget_tag[2].text.split(' ')[1]
budget

'$20,000,000.00'

## scrape source link
- title, url
- url: https://www.themoviedb.org/movie

In [ ]:
url = 'https://www.themoviedb.org/movie'
resp = requests.get(url)
html_doc = BeautifulSoup(resp.text, 'html.parser')

In [ ]:
title_tag = html_doc.find_all('div',{'class':'content'})
# title_tag = html_doc.select('.page_wrapper > div.content')
title_tag = [t.select_one('h2 > a') for t in title_tag ]
# title = title_tag[0].text
# title_tag[0].text
del title_tag[0]
title_tag[0].get('href')

'/movie/460465'

In [ ]:
title_tag

[<a href="/movie/460465" title="Mortal Kombat">Mortal Kombat</a>,
 <a href="/movie/460465" title="Mortal Kombat">Mortal Kombat</a>,
 <a href="/movie/399566" title="Godzilla vs. Kong">Godzilla vs. Kong</a>,
 <a href="/movie/615457" title="Nobody">Nobody</a>,
 <a href="/movie/567189" title="Tom Clancy's Without Remorse">Tom Clancy's Without Remorse</a>,
 <a href="/movie/804435" title="Vanquish">Vanquish</a>,
 <a href="/movie/635302" title="Demon Slayer -Kimetsu no Yaiba- The Movie: Mugen Train">Demon Slayer -Kimetsu no Yaiba- The Movie: Mugen Train</a>,
 <a href="/movie/632357" title="The Unholy">The Unholy</a>,
 <a href="/movie/726684" title="Miraculous World: Shanghai – The Legend of Ladydragon">Miraculous World: Shanghai – The Legend of Ladydragon</a>,
 <a href="/movie/791373" title="Zack Snyder's Justice League">Zack Snyder's Justice League</a>,
 <a href="/movie/813258" title="Monster Pets: A Hotel Transylvania Short">Monster Pets: A Hotel Transylvania Short</a>,
 <a href="/movie/634

## scrape source link with func.
- url: https://www.themoviedb.org/movie

In [ ]:
movies_dict = {
    'title': [],
    'release_date': [],
    'genres': [],
    'runtime':[],
    'url':[]
}

func for each movie details
- get release date, runtime, genres

In [ ]:
def get_relese_date(html_doc):
    release_date_tag = html_doc.select_one('.release')
    release_date = release_date_tag.text.strip()
    return release_date

def get_genres(html_doc):
    genres_tag = html_doc.select_one('.genres')
    genres = genres_tag.text.strip().replace("\xa0", "")
    return genres

def get_runtime(html_doc):
    runtime_tag = html_doc.select_one('.runtime')
    runtime = runtime_tag.text.strip()
    return runtime
    
def get_each_movie_details(url):
    resp = requests.get(url)
    html_doc = BeautifulSoup(resp.text, 'html.parser')
    return get_relese_date(html_doc), get_genres(html_doc), get_runtime(html_doc)


func for single page data scrape
- get title, url
- add all in dict

In [ ]:
def scrape_movies(html_doc):

    a_tag = html_doc.find_all('div',{'class':'content'})
    a_tag = [t.select_one('h2 > a') for t in a_tag ]
    for item in a_tag:
        if item!= None:
            movies_dict['title'].append(item.text)
            url = 'https://www.themoviedb.org/'+item.get('href',None)
            release_date, genres, runtime = get_each_movie_details(url)
            movies_dict['release_date'].append(release_date)
            movies_dict['genres'].append(genres)
            movies_dict[ 'runtime'].append(runtime)
            movies_dict['url'].append(url)



func for get nth page 
- collect each html doc per page

In [ ]:
def get_data(page):
    data = {
    'air_date.gte': '',
    'air_date.lte': '2021-11-09',
    'certification': '',
    'certification_country': 'BD',
    'debug': '',
    'first_air_date.gte': '',
    'first_air_date.lte': '',
    'ott_region': 'AR',
    'page': '1',
    'primary_release_date.gte': '',
    'primary_release_date.lte': '',
    'region': '',
    'release_date.gte': '',
    'release_date.lte': '2021-11-09',
    'show_me': '0',
    'sort_by': 'popularity.desc',
    'vote_average.gte': '0',
    'vote_average.lte': '10',
    'vote_count.gte': '',
    'with_genres': '',
    'with_keywords': '',
    'with_networks': '',
    'with_origin_country': '',
    'with_original_language': '',
    'with_ott_monetization_types': '',
    'with_ott_providers': '',
    'with_release_type': '',
    'with_runtime.gte': '0',
    'with_runtime.lte': '400'
    }
    data['page'] = page
    return data

def scrape_nth_page(pages):
    for page in range(1,pages+1):
        response = requests.post('https://www.themoviedb.org/discover/movie/items', data=get_data(page))
        html_doc = BeautifulSoup(response.text, 'html.parser')
        scrape_movies(html_doc)

In [ ]:
scrape_nth_page(2)

In [ ]:
pd.DataFrame(movies_dict)

,title,release_date,genres,runtime,url
0,Mortal Kombat,04/23/2021 (US),"Action,Fantasy,Adventure,Science Fiction",1h 50m,https://www.themoviedb.org//movie/460465
1,Godzilla vs. Kong,03/24/2021 (TW),"Science Fiction,Action,Drama",1h 53m,https://www.themoviedb.org//movie/399566
2,Nobody,03/26/2021 (US),"Action,Thriller,Crime",1h 32m,https://www.themoviedb.org//movie/615457
3,Tom Clancy's Without Remorse,04/29/2021 (US),"Action,Adventure,Thriller,War",1h 49m,https://www.themoviedb.org//movie/567189
4,Vanquish,04/16/2021 (US),"Action,Crime,Thriller",1h 36m,https://www.themoviedb.org//movie/804435
5,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,10/30/2020 (TW),"Animation,Action,Adventure,Fantasy,Drama",1h 57m,https://www.themoviedb.org//movie/635302
6,The Unholy,04/02/2021 (US),Horror,1h 39m,https://www.themoviedb.org//movie/632357
7,Miraculous World: Shanghai – The Legend of Lad...,04/04/2021 (FR),"Animation,Comedy,Family,Adventure",52m,https://www.themoviedb.org//movie/726684
8,Zack Snyder's Justice League,03/18/2021 (TW),"Action,Adventure,Fantasy,Science Fiction",4h 2m,https://www.themoviedb.org//movie/791373
9,Monster Pets: A Hotel Transylvania Short,04/02/2021 (US),"Animation,Comedy,Fantasy",6m,https://www.themoviedb.org//movie/813258
